# CS 109A/STAT 121A/AC 209A/CSCI E-109A: Homework 4
# Regularization, High Dimensionality, PCA


**Harvard University**<br/>
**Fall 2017**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine

---

### INSTRUCTIONS

- To submit your assignment follow the instructions given in canvas.
- Restart the kernel and run the whole notebook again before you submit. 
- Do not include your name(s) in the notebook even if you are submitting as a group. 
- If you submit individually and you have worked with someone, please include the name of your [one] partner below. 

---

Your partner's name (if you submit separately):

Enrollment Status (109A, 121A, 209A, or E109A):

Import libraries:

In [94]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn.cross_validation import KFold
from sklearn.decomposition import PCA
import statsmodels.regression.linear_model as sm
from statsmodels.regression.linear_model import OLS
import sklearn.preprocessing as Preprocessing
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import Ridge as Ridge_Reg
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn.cross_validation import KFold
from sklearn.decomposition import PCA
import statsmodels.regression.linear_model as sm
from statsmodels.regression.linear_model import OLS
import sklearn.preprocessing as Preprocessing
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
from itertools import combinations

from sklearn.metrics import r2_score
%matplotlib inline

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Continuing Bike Sharing Usage Data

In this homework, we will focus on multiple linear regression, regularization, dealing with high dimensionality, and PCA. We will continue to build regression models for the Capital Bikeshare program in Washington D.C.  See Homework 3 for more information about the data.

*Note: please make sure you use all the processed data from HW 3 Part (a)...you make want to save the data set on your computer and reread the csv/json file here.


In [95]:
train = pd.read_csv('Bikeshare_train.csv', index_col=0)
test = pd.read_csv('Bikeshare_test.csv', index_col=0)
cat_preds = ['season', 'month', 'day_of_week', 'weather']
labels = [['summer', 'fall', 'winter'],['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul','Aug','Sep','Oct','Nov','Dec'], 
          ['mon', 'tue', 'wed','thu', 'fri','sat'], ['cloud', 'light']]

# loop through categorical variables and convert to one-hot encoding
for i,pred in enumerate(cat_preds): 
    dum = pd.get_dummies(train[pred])
    del dum [1]
    dum.columns = labels[i]
    train = train.join(dum)
    del train[pred]

# Repeat the same process for test data
for i,pred in enumerate(cat_preds): 
    dum = pd.get_dummies(test[pred])
    del dum [1]
    dum.columns = labels[i]
    test = test.join(dum)
    del test[pred]
    
# Mean and STD of continuous preidctors based on trainig set
temp_std = train['temp'].std(axis = 0)
temp_mean = train['temp'].mean(axis = 0)
atemp_std = train['atemp'].std(axis = 0)
atemp_mean = train['atemp'].mean(axis = 0)
humidity_std = train['humidity'].std(axis = 0)
humidity_mean = train['humidity'].mean(axis = 0)
wind_std = train['windspeed'].std(axis = 0)
wind_mean = train['windspeed'].mean(axis = 0)

# normalize training data
train['temp'] = (train['temp'] - temp_mean) / temp_std
train['atemp'] = (train['atemp'] - atemp_mean) / atemp_std
train['humidity'] = (train['humidity'] - humidity_mean) / humidity_std
train['windspeed'] = (train['windspeed'] - wind_mean) / wind_std
# normalize test data
test['temp'] = (test['temp'] - temp_mean) / temp_std
test['atemp'] = (test['atemp'] - atemp_mean) / atemp_std
test['humidity'] = (test['humidity'] - humidity_mean) / humidity_std
test['windspeed'] = (test['windspeed'] - wind_mean) / wind_std


In [96]:
y_train = train['count']
# Get data for all predictors
x = list(train.columns)
x.remove('count')
x_train = train[x]
print(x_train.columns)

y_test = test['count']
# Get data for all predictors
x = list(test.columns)
x.remove('count')
x_test = test[x]
print(x_test.columns)

Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'summer', 'fall', 'winter', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'mon', 'tue', 'wed', 'thu', 'fri',
       'sat', 'cloud', 'light'],
      dtype='object')
Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'summer', 'fall', 'winter', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'mon', 'tue', 'wed', 'thu', 'fri',
       'sat', 'cloud', 'light'],
      dtype='object')


In [97]:
train.head()

,holiday,workingday,temp,atemp,humidity,windspeed,count,summer,fall,winter,...,Nov,Dec,mon,tue,wed,thu,fri,sat,cloud,light
0,0.0,1.0,0.623798,0.650106,0.920664,-0.928758,6073.0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0.0,1.0,-0.180310,-0.054759,0.696852,-0.213502,6606.0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
2,0.0,1.0,0.802489,0.851495,-0.448383,0.803926,7363.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0.0,0.0,-1.520492,-1.565182,-0.332113,-0.269099,2431.0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
4,0.0,1.0,0.534453,0.348021,1.975789,-1.199027,1996.0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


## Part (f): Regularization/Penalization Methods

As an alternative to selecting a subset of predictors and fitting a regression model on the subset, one can fit a linear regression model on all predictors, but shrink or regularize the coefficient estimates to make sure that the model does not "overfit" the training set. 

Use the following regularization techniques to fit linear models to the training set:
- Ridge regression
- Lasso regression
    
You may choose the shrikage parameter $\lambda$ from the set $\{10^{-5}, 10^{-4},...,10^{4},10^{5}\}$ using cross-validation. In each case, 

- How do the estimated coefficients compare to or differ from the coefficients estimated by a plain linear regression (without shrikage penalty) in Part (b) fropm HW 3? Is there a difference between coefficients estimated by the two shrinkage methods? If so, give an explantion for the difference.
- List the predictors that are assigned a coefficient value close to 0 (say < 1e-10) by the two methods. How closely do these predictors match the redundant predictors (if any) identified in Part (c) from HW 3?
- Is there a difference in the way Ridge and Lasso regression assign coefficients to the predictors `temp` and `atemp`? If so, explain the reason for the difference.

We next analyze the performance of the two shrinkage methods for different training sample sizes:
- Generate random samples of sizes 100, 150, ..., 400 from the training set. You may use the following code to draw a random sample of a specified size from the training set:

In [122]:
## RS NOTE THIS ISNT USING x VALIDATION...
alpha= [-5, -4, -3, -2, -1, 0, 1, 2,3, 4, 5]
train_r_squared = np.zeros(len(alpha))
test_r_squared = np.zeros(len(alpha))

print("RIDGE")

for i, alpha in enumerate(alpha):
    reg = Ridge_Reg(10**alpha)
    reg.fit(x_train, y_train)
    coefficients = reg.coef_
    # calculate r2 for testing and training sets
    test_r_squared[i] = reg.score(x_test, y_test)
    train_r_squared[i] = reg.score(x_train, y_train)
    print('Ridge')   
    print('Lambda = ', 10**alpha, '  Test R2 =  ', reg.score(x_test, y_test))
    print('Coefficients:', coefficients)
    print('Predictors with non-zero coefficients:', [i for i, item in enumerate(coefficients) if abs(item) > 0])
    print('predictors w/ a coefficient value close to 0:', [i for i, item in enumerate(coefficients) if abs(item) < .0000000001])

RIDGE
Ridge
Lambda =  1e-05   Test R2 =   0.249342182912
Coefficients: [ -468.46678635   124.04186362   925.73195625   312.43496903  -548.49306923
  -255.12261663   898.82426773  1032.87469071  1226.18213967    88.9412038
   239.1839445    333.35559126   -65.80497803  -792.25406882 -1279.97614961
  -776.46482545   405.15341993   486.25723467   112.68717398  -118.83192646
   -60.35988713   -71.53430224   294.26265235   185.0077752    234.81849096
   404.78480066   -16.56641837 -1581.97556793]
Predictors with non-zero coefficients: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
predictors w/ a coefficient value close to 0: []
Ridge
Lambda =  0.0001   Test R2 =   0.249342825815
Coefficients: [ -468.46050266   124.04125062   925.71491642   312.44304299  -548.49415086
  -255.12286539   898.78132931  1032.81274274  1226.14251974    88.95120007
   239.21135196   333.41412149   -65.73729529  -792.17461825 -1279.87845875
  -776.36886386   

In [123]:
alpha= [-5, -4, -3, -2, -1, 0, 1, 2,3, 4, 5]
train_r_squared = np.zeros(len(alpha))
test_r_squared = np.zeros(len(alpha))
print("LASSO")
for i, alpha in enumerate(alpha):
    reg = Lasso_Reg(10**alpha)
    reg.fit(x_train, y_train)
    # calculate r2 for testing and training sets
    test_r_squared[i] = reg.score(x_test, y_test)
    train_r_squared[i] = reg.score(x_train, y_train)
    print("Lasso")
    print('Lambda = ', 10**alpha, '  Test R2 =  ', reg.score(x_test, y_test))
    print('Coefficients:', coefficients)
    print('Predictors with non-zero coefficients:', [i for i, item in enumerate(coefficients) if abs(item) > 0])
    print('predictors w/ a coefficient value close to 0:', [i for i, item in enumerate(coefficients) if abs(item) < .0000000001])

LASSO
Lasso
Lambda =  1e-05   Test R2 =   0.249342144656
Coefficients: [-0.04399607  0.14892471  3.78741153  3.81017076 -0.87386525 -1.3282172
  0.33173557  0.94424731  0.12740058 -0.36766736 -0.227927    0.0369306
  0.23197668  0.20436411  0.24731069  0.27464861  0.46465641  0.24101608
 -0.12217216 -0.31463698 -0.13530467 -0.06345241  0.13201638  0.06221797
  0.08411674  0.03037603 -0.5658001  -0.28563892]
Predictors with non-zero coefficients: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
predictors w/ a coefficient value close to 0: []
Lasso
Lambda =  0.0001   Test R2 =   0.249342443333
Coefficients: [-0.04399607  0.14892471  3.78741153  3.81017076 -0.87386525 -1.3282172
  0.33173557  0.94424731  0.12740058 -0.36766736 -0.227927    0.0369306
  0.23197668  0.20436411  0.24731069  0.27464861  0.46465641  0.24101608
 -0.12217216 -0.31463698 -0.13530467 -0.06345241  0.13201638  0.06221797
  0.08411674  0.03037603 -0.5658001  -0.28

In [38]:
#--------  sample
# A function to select a random sample of size k from the training set
# Input: 
#      x (n x d array of predictors in training data)
#      y (n x 1 array of response variable vals in training data)
#      k (size of sample) 
# Return: 
#      chosen sample of predictors and responses

def sample(x, y, k):
    n = x.shape[0] # No. of training points
    
    # Choose random indices of size 'k'
    subset_ind = np.random.choice(np.arange(n), k)
    
    # Get predictors and reponses with the indices
    x_subset = x[subset_ind, :]
    y_subset = y[subset_ind]
    
    return (x_subset, y_subset)

Generate random samples of sizes 100, 150, ..., 400 from the training set. You may use the following code to draw a random sample of a specified size from the training set:
- Fit linear, Ridge and Lasso regression models to each of the generated sample. In each case, compute the $R^2$ score for the model on the training sample on which it was fitted, and on the test set.
- Repeat the above experiment for 10 random trials/splits, and compute the average train and test $R^2$ across the trials for each training sample size. Also, compute the standard deviation (SD) in each case.
- Make a plot of the mean training $R^2$ scores for the linear, Ridge and Lasso regression methods as a function of the training sample size. Also, show a confidence interval for the mean scores extending from **mean - SD** to **mean + SD**. Make a similar plot for the test $R^2$ scores.

How do the training and test $R^2$ scores compare for the three methods? Give an explanation for your observations. How do the confidence intervals for the estimated $R^2$ change with training sample size? Based on the plots, which of the three methods would you recommend when one needs to fit a regression model using a small training sample?

*Hint:* You may use `sklearn`'s `RidgeCV` and `LassoCV` classes to implement Ridge and Lasso regression. These classes automatically perform cross-validation to tune the parameter $\lambda$ from a given range of values. You may use the `plt.errorbar` function to plot confidence bars for the average $R^2$ scores.

In [133]:
sample_size = list(range(100,450,50))
print(sample_size)


[100, 150, 200, 250, 300, 350, 400]


In [ ]:
reg = Lin_Reg()
reg.fit(x_train, y_train)

# score the model against the test and train sets
test_r_squared_plain = reg.score(x_test, y_test)
train_r_squared_plain = reg.score(x_train, y_train)



In [ ]:
## RS This code should help us plot the work later. needs to be adapted to include Ridge and lasso too. 

def PlotR2(lambdas, test_r_squared_plain, train_r_squared, test_r_squared):
    # Plot train and test R-squared as a function parameter value
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # plain regression R2
    ax.axhline(y = test_r_squared_plain, c='g', label = 'Plain Regression')
    
    # test and train R2
    ax.semilogx(10.0**lambdas, (train_r_squared), c='b', label='Ridge: Train')
    ax.semilogx(10.0**lambdas, (test_r_squared), c='r', label='Ridge: Test')

    # add labels 
    ax.set_xlabel('Regularization parameter $\lambda$')
    ax.set_ylabel(r'$R^2$ score')
    ax.legend(loc = 'best')

    print 'Regression: max R^2 score on training set', max(train_r_squared)
    print 'Regression: max R^2 score on test set', max(test_r_squared)
    plt.show()
#####

your answers here

## Part (g): Polynomial & Interaction Terms

Moving beyond linear models, we will now try to improve the performance of the regression model in Part (b) from HW 3 by including higher-order polynomial and interaction terms. 

- For each continuous predictor $X_j$, include additional polynomial terms $X^2_j$, $X^3_j$, and $X^4_j$, and fit a multiple regression model to the expanded training set. How does the $R^2$ of this model on the test set compare with that of the linear model fitted in Part (b) from HW 3? Using a t-test, find out which of estimated coefficients for the polynomial terms are statistically significant at a significance level of 5%. 

- Fit a multiple linear regression model with additional interaction terms $\mathbb{I}_{month = 12} \times temp$ and $\mathbb{I}_{workingday = 1} \times \mathbb{I}_{weathersit = 1}$ and report the test $R^2$ for the fitted model. How does this compare with the $R^2$ obtained using linear model in Part (b) from HW 3? Are the estimated coefficients for the interaction terms statistically significant at a significance level of 5%?

In [ ]:
#RS: NOTES ARE HERE: https://github.com/cs109/a-2017/blob/40446caa5cea644068c564fb5e2dd06163a838cf/Lectures/Lecture4-IntroRegression/Lecture5_Notebook.ipynb

y_train = train['count'].values,
X_train = train[['temp', 'atemp', 'humidity', 'windspeed']].values

a1= (X_train)
a2= (X_train**2)
a3= (X_train**3)
a4= (X_train**4)
all_as = np.append([a1], [a2])
#all_as = np.append(all_as, a3)
#all_as = np.append(all_as, a4)
y_test = test['count'].values
X_test = test[['temp', 'atemp', 'humidity', 'windspeed']].values



In [ ]:
all_as.shape

In [ ]:
x_train=X_train
# Create the PolynomialFeatures object and specify the number of degrees:
#degree = 2
#poly = PolynomialFeatures(degree)

x_train_poly = poly.fit_transform(all_as)
x_test_poly = poly.fit_transform(x_test)
pd.DataFrame(x_train_poly).shape

# Create a linear regression object
lg = LinearRegression()

# Fit our training data with polynomial features 
final =lg.fit(x_train_poly, y_train)

#final.summary()

#print lg.predict(x_test_poly)
#############
# Obtain coefficients
len(lg.coef_)

#poly.predicted = lg.predict(x_test_poly)
#(poly.predicted)



/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [59]:
gen_cross_terms = PolynomialFeatures(degree=3, interaction_only=True)
cross_terms = gen_cross_terms.fit_transform(X_train)
X_train_with_cross = np.hstack((X_train, cross_terms))
cross_terms = gen_cross_terms.fit_transform(X_test)
X_test_with_cross = np.hstack((X_test, cross_terms))

In [ ]:
"""
""""""def make_features(train_set, test_set, degrees):
    train_dict = {}
    test_dict = {}
    for d in degrees:
        traintestdict={}
        train_dict[d] = PolynomialFeatures(d).fit_transform(train_set.reshape(-1,1))
        test_dict[d] = PolynomialFeatures(d).fit_transform(test_set.reshape(-1,1))
    return train_dict, test_dict

degrees=range(21)
train_dict, test_dict = make_features(x_train, x_test, degrees)

degrees=[1,2,3]
Xtrain= x_train
Xtest = x_test
ytrain = y_train
#for each degree, we now fit on the training set and predict on the test set
#we accumulate the MSE on both sets in error_train and error_test
for d in degrees:#for increasing polynomial degrees 0,1,2...
    Xtrain = train_dict[d]
    Xtest = test_dict[d]
    #set up model
    est = LinearRegression()
    #fit
    est.fit(Xtrain, ytrain)
    #predict
    #your code here""""""

## Part (h): PCA to deal with high dimensionality

We would like to fit a model to include all main effects, polynomial terms up to the $4^{th}$ order, and all interactions between all possible predictors and polynomial terms (not including the interactions between $X^1_j$, $X^2_j$, $X^3_j$, and $X^4_j$ as they would just create higher order polynomial terms).  

- Create an expanded training set including all the desired terms mentioned above.  What are the dimensions of this 'design matrix' of all the predictor variables?   What are the issues with attempting to fit a regression model using all of these predictors?

- Instead of using the usual approaches for model selection, let's instead use principal components analysis (PCA) to fit the model.  First, create the principal component vectors in python (consider: should you normalize first?).  Then fit 5 different regression models: (1) using just the first PCA vector, (2) using the first two PCA vectors, (3) using the first three PCA vectors, etc...  Briefly summarize how these models compare in the training set.

- Use the test set to decide which of the 5 models above is best to predict out of sample.  How does this model compare to the previous models you've fit?  What are the interpretations of this model's coefficients?

In [ ]:
#your code here

your answers here

## Part (i): Beyond Squared Error

We have seen in class that the multiple linear regression method optimizes the Mean Squared Error (MSE) on the training set. Consider the following alternate evaluation metric, referred to as the Root Mean Squared Logarthmic Error (RMSLE):

$$
\sqrt{\frac{1}{n}\sum_{i=1}^n (log(y_i+1) - log(\hat{y}_i+1))^2}.
$$

The *lower* the RMSLE the *better* is the performance of a model. The RMSLE penalizes errors on smaller responses more heavily than errors on larger responses. For example, the RMSLE penalizes a prediction of $\hat{y} = 15$ for a true response of $y=10$ more heavily than a prediction of $\hat{y} = 105$ for a true response of $100$, though the difference in predicted and true responses are the same in both cases. 

This is a natural evaluation metric for bike share demand prediction, as in this application, it is more important that the prediction model is accurate on days where the demand is low (so that the few customers who arrive are served satisfactorily), compared to days on which the demand is high (when it is less damaging to lose out on some customers).

The following code computes the RMSLE for you:

In [2]:
#--------  rmsle
# A function for evaluating Root Mean Squared Logarithmic Error (RMSLE)
# of the linear regression model on a data set
# Input: 
#      y_test (n x 1 array of response variable vals in testing data)
#      y_pred (n x 1 array of response variable vals in testing data)
# Return: 
#      RMSLE (float) 

def rmsle(y, y_pred):     
    # Evaluate sqaured error, against target labels
    # rmsle = \sqrt(1/n \sum_i (log (y[i]+1) - log (y_pred[i]+1))^2)
    rmsle_ = np.sqrt(np.mean(np.square(np.log(y+1) - np.log(y_pred+1))))
    
    return rmsle_

Use the above code to compute the training and test RMSLE for the polynomial regression model you fit in Part (g). 

You are required to develop a strategy to fit a regression model by optimizing the RMSLE on the training set. Give a justification for your proposed approach. Does the model fitted using your approach yield lower train RMSLE than the model in Part (g)? How about the test RMSLE of the new model? 

**Note:** We do not require you to implement a new regression solver for RMSLE. Instead, we ask you to think about ways to use existing built-in functions to fit a model that performs well on RMSLE. Your regression model may use the same polynomial terms used in Part (g).

In [ ]:
# your code here

your answers here

## Part (j): Dealing with Erroneous Labels

Due to occasional system crashes, some of the bike counts reported in the data set have been recorded manually. These counts are not very unreliable and are prone to errors. It is known that roughly 5% of the labels in the training set are erroneous (i.e. can be arbitrarily different from the true counts), while all the labels in the test set were confirmed to be accurate. Unfortunately, the identities of the erroneous records in the training set are not available. Can this information about presence of 5% errors in the training set labels (without details about the specific identities of the erroneous rows) be used to improve the performance of the model in Part (g)? Note that we are interested in improving the $R^2$ performance of the model on the test set (not the training $R^2$ score). 

As a final task, we require you to come up with a strategy to fit a regression model, taking into account the errors in the training set labels. Explain the intuition behind your approach (we do not expect a detailed mathematical justification). Use your approach to fit a regression model on the training set, and compare its test $R^2$ with the model in Part (g).

**Note:** Again, we do not require you to implement a new regression solver for handling erroneous labels. It is sufficient that you to come up with an approach that uses existing built-in functions. Your regression model may use the same polynomial terms used in Part (g).

In [ ]:
# your code here

In [ ]:
your answers here

---
--_